<a href="https://colab.research.google.com/github/JoshuaOgbe/JoshuaOgbe/blob/main/HCUP_Substance_Abuse_Predict_Admitted_Patients_Length_of_Stay1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Steps: 
# 1. Install libraries
# 2. Import all libraries
# 4. Set parameters for FS, MT, Preprocess, fairness metrics etc. 
# 3. Preprocessing
# 4. Feature selection
# 5. Model trainign
# 6. Model evaulation
# 7. Unfairness metrics calculation of raw model. 
# 8. Mitigating the unfairness
# 9. Unfariness metrics for mitigated model. 

In [ ]:
# Author: Kshitij Singh
# Subject: Code for LoS prediction for substance abuse patients. Evaluating fairness metrics and mitigating unfairness in models. 

In [ ]:
# 1. Make all dataset consistent. 
# 1. Run the mitigation code. Save results for all. 
# 2. Do it for Xinyu's dataset and save the result. 
# 3. Send the code and result by email. 

In [ ]:
!pip3 install fairlearn
!pip3 install pytorch_tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 7.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 631 kB/s 


In [ ]:
# Author: Kshitij Singh


# Imports
import pandas as pd
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble._bagging import BaggingClassifier
from sklearn.ensemble._forest import ExtraTreesClassifier

from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble._bagging import BaggingClassifier
from sklearn.metrics import confusion_matrix, fbeta_score

from fairlearn.metrics import MetricFrame
from fairlearn.metrics import selection_rate, count
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.reductions import ExponentiatedGradient, ClassificationMoment, Moment, DemographicParity, GridSearch
from fairlearn.metrics import equalized_odds_difference, equalized_odds_ratio, false_negative_rate_difference, false_positive_rate_difference, true_positive_rate_difference, true_negative_rate_difference, demographic_parity_ratio, demographic_parity_difference

# Suppress warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
# Setting parameters

alpha = 0.1 # Statistical significance for chi2 test. 
k = 3 # Minimum number of feature selection method selecting a given feature. 

models_to_train = [
    # LogisticRegression, 
    RandomForestClassifier, 
    # DecisionTreeClassifier, 
    XGBClassifier, 
    # BaggingClassifier
]


unfairness_metrics = [
    selection_rate, 
    fbeta_score, 
    equalized_odds_difference, 
    equalized_odds_ratio
]
model_evaulation = [
    accuracy_score, 
    recall_score, 
    precision_score, 
    f1_score 
]

demographic_features = [
    'RACE', 
    'FEMALE', 
    'MARITALSTATUSUB04', 
    'ZIPINC_QRTL', 
    'AGE'
    
]

financial_features = [
    'MEDINCSTQ',
     'PAY1',
     'Homeless'
]

medical_features = ['I10_NDX']

# Choose the appropriate variable for selected features depending on the dataset being used. 
# Keep all categorical features numeric. 
selected_features_florida = [
 'DRG',
 'DHOUR',
 'PAY1_X',
 'MDC',
 'I10_NDX',
 'EDHOUR',
 'AHOUR',
 'AGE',
 'PSTCO2',
 'I10_NPR',
 'DaysToEvent',
 'ATYPE',
 'YEAR',
 'TRAN_OUT',
 'PointOfOriginUB04',
 'PSTCO',
 'POA_Hosp_Edit1',
 'P7EDSRC_X',
 'HCUP_ED',
 'EDYEAR',
 'DXPOA2',
 'DISPUB04',
 'AWEEKEND', 
 'RACE', 
 'FEMALE', 
 'AGE', 
 'ZIPINC_QRTL']

selected_features_maryland = ['AGE',
'ATYPE',
'AWEEKEND',
'BWT',
'DRG',
'DISPUNIFORM',
'DISP_X',
'DSHOSPID',
'DaysBurnUnit',
'DaysCCU',
'DaysICU',
'DaysNICU',
'DaysPICU',
'DaysShockUnit',
'DaysToEvent',
'FEMALE',
'HISPANIC',
'Homeless',
'HospitalUnit',
'LOS_status',
'MARITALSTATUSUB04',
'MEDINCSTQ',
'PSTATE',
'PSTCO',
'PointOfOriginUB04',
'RACE',
'READMIT',
'P7EDSRC_X',
'PAY1',
'PAY2',
'PAY3',
'TRAN_IN',
'TRAN_OUT',
'VisitLink',
'ZIPINC_QRTL',
'ZIP',
'PrimLang',
'AYEAR',
    ]

selected_features_arizona = ['AGE',
            'ATYPE', 'AWEEKEND', 'DISPUNIFORM',
            'HCUP_ED', 'I10_NDX','I10_NPR','MDC',
            'MDC_NoPOA','PRDAY2','PRMONTH1','TRAN_OUT',
            'ATOD','DTOD', 'TRAN_IN', 'AHOUR', 'AMONTH',
            'DHOUR', 'DMONTH', 'DXPOA1', 'FEMALE',
            'MEDINCSTQ', 'HISPANIC', 'Homeless',
            'MARITALSTATUSUB04','PAY1','RACE','ZIPINC_QRTL']

selected_features_teds = ['AGE', 'ALCDRUG', 'ALCFLG', 'ARRESTS', 'DAYWAIT',
                     'DETCRIM', 'DISYR', 'DIVISION', 'DSMCRIT', 'EDUC', 
                     'EMPLOY', 'EMPLOY_D', 'ETHNIC', 'FREQ1', 'FREQ1_D', 
                     'FREQ2', 'FREQ3_D', 'FREQ_ATND_SELF_HELP', 'FREQ_ATND_SELF_HELP_D', 
                     'FRSTUSE1', 'FRSTUSE2', 'FEMALE', 'HERFLG', 'HLTHINS', 
                     'MARFLG', 'MARSTAT', 'METHUSE', 'NOPRIOR', 'OPSYNFLG', 
                     'PREG', 'ZIPINC_QRTL', 'PSOURCE', 'PSYPROB',
                     'RACE', 'REASON', 'REGION', 'SERVICES', 'SERVICES_D', 
                     'STFIPS', 'SUB1_D', 'SUB2_D', 'SUB3', 'SUB3_D', 'PAY1_X'] 

file_path = '/content/drive/MyDrive/substance_only.csv'
output_path = '/content/'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Inconsistency in datasets: 
# 1. In Maryland dataset target column is called LoS_status. 
# 1. In Maryland LOS_status is already encoded. 
# 2. In Arizona the target column is called LOS_X
# 2. In Arizona it is AGE and not AGE_group
# 2. In Arizona encoding is 'Long Stay' or 'Short Stay' whereas all others have 0 and 1. 
# 2. In Arizona a lot of columns use String for classes whereas other daatsets use label encoding or numerical values for encoding. For tree based algorithm one-hot encoding reduces their accuracy. 
# 3. Demographic feature names are not consistent across datasets. Following is the name of the features: 
# Insurance - Pay1_X
# Age - AGE
# Race - RACE
# Gender - FEMALE
# Income - ZIPINC_QRTL
# 4. In TEDS there is no ZIPINC_QRTL, No insurance information. 
# 5. Handling blank. Some dataset it is replaced with 99999 and in some with 100. 

In [ ]:
from pandas.core.groupby.generic import DataFrame
from pandas.io.formats.format import DataFrameRenderer
# Preprocessing
# To do: 
# 1. Test reading data for Maryland and other dataset. 
# 2. Convert age into multiple categories 0 - 15, 15 - 25, 25 - 35. 

def read_data(file_path):
    """
    Return pandas dataframe. 
    """
    return pd.read_csv(file_path)

def clean_and_preprocess(df, columns_to_select, select_all=False):
    """
    Returns X_train, y_train, X_test, y_test. 
    """
    # To do: 
    # 1. Select only required columns for a given dataset. 
    # 2. Add option to select required columns or select all columns. 
    # df.drop(columns=['Unnamed: 0','HCUP_OS','LOS_X','DRG_NoPOA','DISPUNIFORM','DISP_X','MDNUM1_R','MDNUM2_R', 'MDNUM3_R','POA_Hosp_Edit2', 'POA_Hosp_Edit3',
        # 'POA_Hosp_Edit3_Value','MDC_NoPOA','HospitalUnit','PointOfOrigin_X', 'POA_Disch_Edit2','RACE_EDIT','RACE_X','TOTCHG_X','ZIP3', 'TOTCHG'],inplace=True)
    df.drop(df.index[df['FEMALE'] == 2], inplace=True)
    df.drop(df.index[df['I10_NDX'] == 24], inplace=True)

    df.dropna(inplace=True)
    def fillSpace(x):
        if x ==' ':
            return 999999
        else:
            return x
        
    for i in df.columns:
        df[i]=df[i].apply(fillSpace)

    # Label encoding
    # le = preprocessing.LabelEncoder()
    # for i in ['DXPOA1','DXPOA2','P7EDSRC_X','PAY1_X']:
    #     df[i]=df[i].astype(str)
    # for i in ['DXPOA1','DXPOA2','P7EDSRC_X','PAY1_X']:
    #     df[i]=le.fit_transform(df[i])
    # df['PointOfOriginUB04']=df['PointOfOriginUB04'].astype(str)
    # df['PointOfOriginUB04']=le.fit_transform(df['PointOfOriginUB04'])


    def los_category(x):
        return x
        if x <= 5:
            return 0

        return 1
    df['LOS_category']=df['LOS_X'].apply(los_category)

    icd_codes=["F10","F11","F12","F13","F14","F15","F16","F17","F18","F19"]

    # df['Admit_DX_Presence']=df['I10_DX_Admitting'].str.contains("|".join(icd_codes), case=False, na = False)
    # df['First_DX_Presence']=df['I10_DX1'].str.contains("|".join(icd_codes), case=False, na = False)
    # df['Second_DX_Presence']=df['I10_DX2'].str.contains("|".join(icd_codes), case=False, na = False)


    # df['Admit_DX_Presence']=df['Admit_DX_Presence'].apply(lambda x: 0 if x == False else 1)
    # df['First_DX_Presence']=df['First_DX_Presence'].apply(lambda x: 0 if x == False else 1)
    # df['Second_DX_Presence']=df['Second_DX_Presence'].apply(lambda x: 0 if x== False else 1)

    # data_hcup.ZIPINC_QRTL = data_hcup.ZIPINC_QRTL.apply(lambda x: int(x))
    def get_race(x):
        if i == 1:
            return 'white'
        elif i == 2:
            return 'black'
        elif i == 3:
            return 'hispanic'
        elif i == 4:
            return 'asian'
        else:
            return 'others'

    df = df[df.ZIPINC_QRTL != '999999']
    df = df[df.AGE != 0]
    df['AGE'] = df['AGE'].apply(lambda x: 0 if x < 25 else 1)

    X=df[selected_features_arizona]
    y=df['LOS_category']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    return X_train, X_test, y_train, y_test, X, y, df
    


In [ ]:
df=pd.read_csv(file_path)
# df.drop(df.index[df['FEMALE'] == 2], inplace=True)

# Doing this for TEDS
# df.rename(columns={"PRIMPAY": 'PAY1_X', 'PRIMINC': 'ZIPINC_QRTL', 'GENDER': 'FEMALE'}, inplace=True)
# X_train, X_test, y_train, y_test, X, y, df = clean_and_preprocess(df, selected_features_arizona)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

df = df.astype({'AHOUR':'category', 'ATYPE':'category', 'AMONTH':'category', 'AWEEKEND':'category', 
                'DHOUR':'category', 'DIED':'category', 'DISPUNIFORM':'category', 'DMONTH':'category', 
                'DXPOA1':'category', 'DXPOA2':'category', 'FEMALE':'category', 'HCUP_ED':'category',
                'HISPANIC':'category', 'Homeless':'category','I10_BIRTH':'category', 'I10_DELIVERY':'category',
                'I10_NDX':'category','I10_NPR':'category', 'I10_PROCTYPE':'category',
                'LOS_X':'category',
                'MARITALSTATUSUB04':'category',
                'MDC':'category','MDC_NoPOA':'category', 'MEDINCSTQ':'category', 'PAY1':'category', 'PL_CBSA':'category', 
                'PL_NCHS':'category', 'PL_RUCC':'category','PL_UIC':'category', 'PL_UR_CAT4':'category',
                'PRDAY1':'category','PRDAY2':'category', 'PointOfOriginUB04':'category', 'SPLIT_IPED':'category',
                'PRMONTH1':'category','ATOD':'category','DTOD':'category','MDBOARD1':'category','MDBOARD2':'category',
                'PRDAY1':'category', 'RACE':'category', 'TRAN_IN':'category', 'TRAN_OUT':'category','ZIPINC_QRTL':'category',
                'PRMONTH1':'category','PRMONTH2':'category'})


Cat = df.select_dtypes(include = "category").columns
# print (Cat)

le = LabelEncoder()

for feat in Cat:
    df[feat] = le.fit_transform(df[feat].astype(str))

In [ ]:
X_train, X_test, y_train, y_test, X, y, df = clean_and_preprocess(df, selected_features_arizona)

In [ ]:
# # DO NOT RUN THIS. FEATURES ARE ALREADY SELECTED. 
# # Feature selection

# def select_features(X, y, k=3):
#     """
#     Returns a list of features selected by atleast k feature selection algorithms. 
#     """
#     # Chi2 selection
#     selector = SelectKBest(chi2, k=46)
#     selector.fit(X, y)
#     cols = selector.get_support(indices=True)
#     features_df_new = X.iloc[:,cols]

#     alpha = 0.05
#     chi_support = [j for i, j in zip(selector.pvalues_, cols) if i < alpha]

#     # RFE


#     # Pearson correlation
#     def cor_selector(X, y,num_feats):
#         cor_list = []
#         feature_name = X.columns.tolist()
#         # calculate the correlation with y for each feature
#         for i in X.columns.tolist():
#             cor = np.corrcoef(X[i], y)[0, 1]
#             cor_list.append(cor)
#         # replace NaN with 0
#         cor_list = [0 if np.isnan(i) else i for i in cor_list]
#         # feature name
#         cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
#         # feature selection? 0 for not select, 1 for select
#         cor_support = [True if i in cor_feature else False for i in feature_name]
#         return cor_support, cor_feature, cor_list
#     cor_support, cor_feature, cor_list = cor_selector(X, y, 20)
#     print(str(len(cor_feature)), 'selected features')


#     # RFE
#     # Selecting optimal number of features
#     feature_count = []
#     for i in range(2, len(X_train.columns)):
#         rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=i, step=10, verbose=5)
#         rfe_selector.fit(X, y)
#         accuracy_s = accuracy_score(rfe_selector.predict(X_test), y_test)
#         rfe_support = rfe_selector.get_support()
#         rfe_feature = X.loc[:,rfe_support].columns.tolist()
#         feature_count.append((i, accuracy_s, rfe_feature))
#         print(str(len(rfe_feature)), 'selected features')
    
#     # Logistic Regression with L1 regularization
#     embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l1", solver="liblinear"), max_features=46)
#     embeded_lr_selector.fit(X, y)

#     embeded_lr_support = embeded_lr_selector.get_support()
#     embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
#     print(str(len(embeded_lr_feature)), 'selected features')

#     # Feature selection using LightGBM model
#     lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
#                 reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

#     embeded_lgb_selector = SelectFromModel(lgbc, max_features=46)
#     embeded_lgb_selector.fit(X, y)

#     embeded_lgb_support = embeded_lgb_selector.get_support()
#     embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
#     print(str(len(embeded_lgb_feature)), 'selected features')

#     # put all selection together
#     feature_selection_df = pd.DataFrame({'Feature':X.columns, 'Pearson':cor_support,
#                                          'Chi-2':chi_support, 'RFE':rfe_support,
#                                          'Logistics':embeded_lr_support,
#                                         'Random Forest':embeded_rf_support,
#                                          'LightGBM':embeded_lgb_support})
#     # count the selected times for each feature
#     feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
#     # display the top 100
#     feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
#     feature_selection_df.index = range(1, len(feature_selection_df)+1)
#     feature_selection_df.head(num_feats)

#     # Save the results
#     feature_selection_df.to_csv("FeatureSelected.csv", index=False)

#     return

In [ ]:
# Model training
from sklearn.metrics import classification_report
def train_models_and_evaluate(X_train, y_train, X_test, y_test,
                              selected_features, 
                              models=models_to_train,
                              ):
    """
    Trains models, returns evaulation metrics and also saves the results 
    to a csv file in the local directory. 
    Returns a list of train model instances. 
    """
    df_metrics = pd.DataFrame(columns=['model', 'accuracy', 'precision', 'recall', 'f1'])
    trained_models = []
    X_train = X_train[selected_features_arizona]
    X_test = X_test[selected_features_arizona]
    for m in models:
        model = m()
        print(f"Fitting {model}...")
        model.fit(X_train, y_train)
        print(f"Predicting for {model}...")
        y_pred = model.predict(X_test)
        target_names=['0','1']
        print(classification_report(y_test, y_pred, target_names=target_names))
        # accuracy = accuracy_score(y_test, y_pred)
        # precision = precision_score(y_pred, y_test)
        # f1 = f1_score(y_pred, y_test)
        # recall = recall_score(y_pred, y_test)
        # print(str(m), accuracy, precision, recall, f1)
        df_metrics.loc[len(df_metrics.index)] = [str(m), 'accuracy', 'precision', 'recall', 'f1']
        trained_models.append(model)
    
    df_metrics.to_csv("models.csv", index=False)
    return trained_models



In [ ]:
# selected_features = list(set([i.strip() for i in selected_features_maryland]))
ms = train_models_and_evaluate(X_train, y_train, X_test, y_test, selected_features_arizona)

Fitting RandomForestClassifier()...
Predicting for RandomForestClassifier()...
              precision    recall  f1-score   support

           0       0.73      0.67      0.70     19200
           1       0.82      0.85      0.84     32752

    accuracy                           0.79     51952
   macro avg       0.77      0.76      0.77     51952
weighted avg       0.79      0.79      0.79     51952

Fitting XGBClassifier()...
Predicting for XGBClassifier()...
              precision    recall  f1-score   support

           0       0.73      0.66      0.69     19200
           1       0.81      0.86      0.83     32752

    accuracy                           0.78     51952
   macro avg       0.77      0.76      0.76     51952
weighted avg       0.78      0.78      0.78     51952



In [ ]:
# Unfariness metrics


import sklearn.metrics as skm
import functools
from fairlearn.metrics import equalized_odds_difference, equalized_odds_ratio

fbeta_06 = functools.partial(skm.fbeta_score, beta=0.6)
fbeta_04 = functools.partial(skm.fbeta_score, beta=0.4)
fbeta_05 = functools.partial(skm.fbeta_score, beta=0.5)
metric_fns = {'selection_rate': selection_rate,
              'fbeta_06': fbeta_06,
              'fbeta_04': fbeta_04,
              'fbeta_05': fbeta_05,
              'count': count}


def unfairness_metrics_evaluation(models, X_test, y_test, prefix="unmitigated"):
    """
    Saves the unfairness metrics for all models in a csv file. 
    Returns a list of dictionaries with key as model object and values 
    as a dictionary of unfairness metrics and their values. 
    """
    df_unfairness_metrics = pd.DataFrame(columns=['categories',
                                                  'demographic_feature',
                                                  'model', 
                                                  'selection_rate', 
                                                  'fbeta_06',
                                                  'fbeta_04',
                                                  'fbeta_05',
                                                  'count'])
    df_equalized_odds = pd.DataFrame(columns=['demographic_feature',
                                              'model',
                                              'equalized_odds_difference', 
                                              'equalized_odds_ratio', 
                                              'demographic_parity_ratio',
                                              'demographic_parity_difference', 
                                              'false_positive_rate_difference', 
                                                    'false_negative_rate_difference',  
                                                    'true_positive_rate_difference', 
                                                     'true_negative_rate_difference', 
                                                    ])
    
    df_overall_model_metrics = pd.DataFrame(columns=['model',
                                                    'false_positive_rate_difference', 
                                                    'false_negative_rate_difference',  
                                                    'true_positive_rate_difference', 
                                                     'true_negative_rate_difference', 
                                                    'overall_balance_error_rate'])
    for m in models:
        y_pred = m.predict(X_test[selected_features_arizona])
        metrics = []
        for d in demographic_features:
            print("Model:", m)
            print("Demographic feature", d)
            results = MetricFrame(metrics=metric_fns,
                                y_true=y_test,
                                y_pred=y_pred,
                                sensitive_features=X_test[[d]])
            results = results.by_group.reset_index()
            print('result')
            results.rename({str(d): 'categories'}, inplace=True)
            results['model'] = str(m)
            print("result2")
            results['demographic_feature'] = str(d)
            print(results)
            df_unfairness_metrics = df_unfairness_metrics.append(results, ignore_index=True)
            print('result1')
            equalized_odds_diff = equalized_odds_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            equalized_odds_r = equalized_odds_ratio(y_test, y_pred, sensitive_features=X_test[[d]])
            demographic_parity_r = demographic_parity_ratio(y_test, y_pred, sensitive_features=X_test[[d]])
            demographic_parity_diff = demographic_parity_difference(y_test, y_pred, sensitive_features=X_test[[d]])

            print("result4")
            false_positive_rate_diff = false_positive_rate_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            false_negative_rate_diff = false_negative_rate_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            true_positive_rate_diff = true_positive_rate_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            true_negative_rate_diff = true_negative_rate_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            # overall_balance_error = overall_balance_error_rate(y_test, y_pred, sensitive_features=X_test[d])
            print("result5")
            df_equalized_odds = df_equalized_odds.append(pd.DataFrame([[str(d), str(m), 
                                                   equalized_odds_difference, 
                                                   equalized_odds_ratio, 
                                                   demographic_parity_ratio, 
                                                   demographic_parity_difference, 
                                                   false_positive_rate_diff, 
                                                      false_negative_rate_diff, 
                                                      true_positive_rate_diff, 
                                                      true_negative_rate_diff]]), 
                                     ignore_index=True)
            print("result6")

            # To do: 
            # 1. Add true positive rate difference
            # 2. Add false positive rate difference
    
    df_equalized_odds.to_csv(f"{prefix}_Equalized_odds.csv", index=False)
    # df_overall_model_metrics.to_csv("Overall_metrics.csv", index=False)
    df_unfairness_metrics.to_csv(f"{prefix}_Unfairness_metrics.csv", index=False)
    return





In [ ]:
unfairness_metrics_evaluation(ms, X_test[selected_features_arizona], y_test)

Model: RandomForestClassifier()
Demographic feature RACE
result
result2
   RACE selection_rate  fbeta_06  fbeta_04  fbeta_05  count  \
0     0        0.62309  0.795067  0.795341  0.795207    589   
1     1       0.625166  0.816686  0.814791  0.815718   3767   
2     2       0.657422  0.837268  0.833846  0.835518   8293   
3     3       0.685144  0.858783  0.856663    0.8577    451   
4     4       0.676347  0.828844  0.821046  0.824845   2858   
5     5          0.825  0.777143  0.752432  0.764331     40   
6     6       0.660928  0.825209  0.820393  0.822744  35954   

                      model demographic_feature  
0  RandomForestClassifier()                RACE  
1  RandomForestClassifier()                RACE  
2  RandomForestClassifier()                RACE  
3  RandomForestClassifier()                RACE  
4  RandomForestClassifier()                RACE  
5  RandomForestClassifier()                RACE  
6  RandomForestClassifier()                RACE  
result1
result4
result5

# Evaluating Unfairness Metrics in each Feature


In [ ]:
X=df[selected_features_arizona]
y = df['LOS_category']
A = df['FEMALE']
B = df['MARITALSTATUSUB04']
C = df['AGE']
D = df['RACE']
E = df['ZIPINC_QRTL']
F = df['MEDINCSTQ']
G = df['Homeless']
F2 = df['PAY1']
H = df['I10_NDX']
A1 = df[['FEMALE', 'RACE']]
# 'MARITALSTATUSUB04','PAY1','RACE','ZIPINC_QRTL'
X_train, X_test, y_train, y_test, A_train, A_test, B_train, B_test,C_train, C_test, D_train, D_test, E_train, E_test, F_train, F_test, G_train, G_test, H_train,H_test, F2_train,F2_test, A1_train,A1_test, = train_test_split(df[selected_features_arizona], 
                                                                     df['LOS_category'],
                                                                     A,B,C,D,E,F,G,H,F2,A1,
                                                                     stratify=df['LOS_category'],
                                                                     test_size=0.3)

In [ ]:
#Using XGBoost Classifier

In [ ]:
# # Fairlearn algorithms and utils
# from fairlearn.postprocessing import ThresholdOptimizer
# from fairlearn.reductions import GridSearch, EqualizedOdds

# Metrics
from fairlearn.metrics import (
    MetricFrame,
    selection_rate, demographic_parity_difference, demographic_parity_ratio,
    false_positive_rate, false_negative_rate,
    false_positive_rate_difference, false_negative_rate_difference,
    equalized_odds_difference)
from sklearn.metrics import balanced_accuracy_score, roc_auc_score

#Using XGBoost Classifier

In [ ]:
#Model Training 
clf = XGBClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)
y_pred = pd.DataFrame(clf.predict(X_test),columns=["Passed"])
target_names=['0','1']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           0       0.72      0.68      0.70     19202
           1       0.82      0.84      0.83     32750

    accuracy                           0.78     51952
   macro avg       0.77      0.76      0.76     51952
weighted avg       0.78      0.78      0.78     51952



In [ ]:
test_scores = clf.predict_proba(X_test)[:, 1]
test_preds = (test_scores >= np.mean(y_train)) * 1
# Train AUC
roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1])

0.8453368568732555

# Female


In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=A_test)

mf.by_group

,FPR,FNR
FEMALE,,
0,0.24591,0.212422
1,0.215576,0.247982


In [ ]:
# Helper functions
def get_metrics_df(models_dict, y_true, group):
    metrics_dict = {
        "Overall selection rate": (
            lambda x: selection_rate(y_true, x), True),
        "Demographic parity difference": (
            lambda x: demographic_parity_difference(y_true, x, sensitive_features=group), True),
        "Demographic parity ratio": (
            lambda x: demographic_parity_ratio(y_true, x, sensitive_features=group), True),
        "------": (lambda x: "", True),
        "Overall balanced error rate": (
            lambda x: 1-balanced_accuracy_score(y_true, x), True),
        "Balanced error rate difference": (
            lambda x: MetricFrame(metrics=balanced_accuracy_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), True),
        " ------": (lambda x: "", True),
        "False positive rate difference": (
            lambda x: false_positive_rate_difference(y_true, x, sensitive_features=group), True),
         " ------": (lambda x: "", True),
       
        "False negative rate difference": (
            lambda x: false_negative_rate_difference(y_true, x, sensitive_features=group), True),
        "Equalized odds difference": (
            lambda x: equalized_odds_difference(y_true, x, sensitive_features=group), True),
        "  ------": (lambda x: "", True),
        "Overall AUC": (
            lambda x: roc_auc_score(y_true, x), False),
        "AUC difference": (
            lambda x: MetricFrame(metrics=roc_auc_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), False),
    }
    df_dict = {}
    for metric_name, (metric_func, use_preds) in metrics_dict.items():
        df_dict[metric_name] = [metric_func(preds) if use_preds else metric_func(scores) 
                                for model_name, (preds, scores) in models_dict.items()]
    return pd.DataFrame.from_dict(df_dict, orient="index", columns=models_dict.keys())

# Female

In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, A_test)

,Unmitigated
Overall selection rate,0.567568
Demographic parity difference,0.049094
Demographic parity ratio,0.917698
------,
Overall balanced error rate,0.230227
Balanced error rate difference,0.002612
------,
False positive rate difference,0.030335
False negative rate difference,0.03556
Equalized odds difference,0.03556


# Marital Status

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=B_test)

mf.by_group

,FPR,FNR
MARITALSTATUSUB04,,
0,0.234483,0.24881
1,0.308585,0.184626
2,0.189427,0.243697
3,0.274131,0.215827
4,0.19324,0.2543
5,0.286139,0.232033


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, B_test)

,Unmitigated
Overall selection rate,0.567568
Demographic parity difference,0.13435
Demographic parity ratio,0.796567
------,
Overall balanced error rate,0.230227
Balanced error rate difference,0.042523
------,
False positive rate difference,0.119157
False negative rate difference,0.069675
Equalized odds difference,0.119157


# AGE

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=C_test)

mf.by_group

,FPR,FNR
AGE,,
0,0.107322,0.338392
1,0.241371,0.224524


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, C_test)

,Unmitigated
Overall selection rate,0.567568
Demographic parity difference,0.169826
Demographic parity ratio,0.708231
------,
Overall balanced error rate,0.230227
Balanced error rate difference,0.010091
------,
False positive rate difference,0.134049
False negative rate difference,0.113868
Equalized odds difference,0.134049


## RACE

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=D_test)

mf.by_group


,FPR,FNR
RACE,,
0,0.164502,0.302857
1,0.210454,0.233291
2,0.21742,0.224053
3,0.264286,0.236527
4,0.223633,0.214127
5,0.380952,0.083333
6,0.232328,0.235585


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, D_test)

,Unmitigated
Overall selection rate,0.567568
Demographic parity difference,0.181297
Demographic parity ratio,0.728055
------,
Overall balanced error rate,0.230227
Balanced error rate difference,0.031527
------,
False positive rate difference,0.21645
False negative rate difference,0.219524
Equalized odds difference,0.219524


# ZIPPINC-QRTL

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=E_test)

mf.by_group

,FPR,FNR
ZIPINC_QRTL,,
0,0.223606,0.234464
1,0.241944,0.22981
2,0.216009,0.234124
3,0.236292,0.217872
4,0.201461,0.267281


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, E_test)

,Unmitigated
Overall selection rate,0.567568
Demographic parity difference,0.059257
Demographic parity ratio,0.898967
------,
Overall balanced error rate,0.230227
Balanced error rate difference,0.01081
------,
False positive rate difference,0.040483
False negative rate difference,0.049409
Equalized odds difference,0.049409


# Financial Group 

# MEDINCSTQ

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=F_test)

mf.by_group

,FPR,FNR
MEDINCSTQ,,
0,0.214603,0.238981
1,0.249679,0.222049
2,0.231437,0.23562
3,0.224138,0.222969
4,0.0,0.0
5,0.202306,0.268164


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, F_test)

,Unmitigated
Overall selection rate,0.567568
Demographic parity difference,0.061043
Demographic parity ratio,0.89622
------,
Overall balanced error rate,0.230227
Balanced error rate difference,0.235864
------,
False positive rate difference,0.249679
False negative rate difference,0.268164
Equalized odds difference,0.268164


# Homeless

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=G_test)

mf.by_group

,FPR,FNR
Homeless,,
0,0.167364,0.339806
1,0.228986,0.230946


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, G_test)

,Unmitigated
Overall selection rate,0.567568
Demographic parity difference,0.124948
Demographic parity ratio,0.780871
------,
Overall balanced error rate,0.230227
Balanced error rate difference,0.023619
------,
False positive rate difference,0.061622
False negative rate difference,0.108859
Equalized odds difference,0.108859


# PAY1

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=F2_test)

mf.by_group

,FPR,FNR
PAY1,,
0,0.244444,0.245364
1,0.212056,0.237097
2,0.218193,0.219428
3,0.285398,0.160656
4,0.2,0.388889
5,0.285924,0.207168


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, F2_test)

,Unmitigated
Overall selection rate,0.567568
Demographic parity difference,0.236602
Demographic parity ratio,0.656897
------,
Overall balanced error rate,0.230227
Balanced error rate difference,0.075634
------,
False positive rate difference,0.085924
False negative rate difference,0.228233
Equalized odds difference,0.228233


# Female and Race

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=A1_test)

mf.by_group

FPR       FNR
FEMALE RACE                    
0      0     0.133333      0.24
       1     0.246654   0.20733
       2      0.22816   0.19574
       3     0.297872  0.247934
       4     0.258427  0.196051
       5     0.333333      0.25
       6     0.249188  0.216606
1      0     0.179487  0.337778
       1     0.190118  0.251076
       2     0.212285  0.240931
       3     0.247312  0.230047
       4      0.20509  0.225895
       5     0.388889      0.05
       6     0.220324  0.250394

In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, A1_test)

,Unmitigated
Overall selection rate,0.567568
Demographic parity difference,0.219644
Demographic parity ratio,0.678982
------,
Overall balanced error rate,0.230227
Balanced error rate difference,0.105
------,
False positive rate difference,0.255556
False negative rate difference,0.287778
Equalized odds difference,0.287778


# MEDICAL GROUP

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=H_test)

mf.by_group

,FPR,FNR
I10_NDX,,
0,0.666667,0.103448
1,0.20904,0.168696
2,0.258763,0.144475
3,0.243184,0.179241
4,0.302997,0.137213
5,0.351077,0.154324
6,0.378897,0.143593
7,0.336842,0.194072
8,0.343795,0.189354


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, H_test)

,Unmitigated
Overall selection rate,0.568236
Demographic parity difference,0.713534
Demographic parity ratio,0.152679
------,
Overall balanced error rate,0.229062
Balanced error rate difference,0.226786
------,
False positive rate difference,0.622711
False negative rate difference,0.714286
Equalized odds difference,0.714286


# Mitigating Equalized Odds Difference in RACE feature  with Postprocessing algorithm ThresholdOptimizer


In [ ]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)

balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
D_train_balanced = D_train.loc[pp_train_idx]

In [ ]:
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=D_train_balanced)

/usr/local/lib/python3.7/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:275: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  FutureWarning,


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(learning_rate=1.0, max_depth=1),
                   prefit=True)

In [ ]:

postprocess_preds = postprocess_est.predict(X_test, sensitive_features=D_test)

In [ ]:
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=D_train_balanced)
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=D_test)
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, D_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.574266,0.743105
Demographic parity difference,0.206139,0.064001
Demographic parity ratio,0.721713,0.919627
------,,
Overall balanced error rate,0.226029,0.355127
Balanced error rate difference,0.107709,0.047085
------,,
False positive rate difference,0.290393,0.055776
False negative rate difference,0.11326,0.074912
Equalized odds difference,0.290393,0.074912


# Mitigating Equalized Odds Difference in MEDINCSTQ feature  with Postprocessing algorithm ThresholdOptimizer

In [ ]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)

balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
F_train_balanced = F_train.loc[pp_train_idx]

In [ ]:
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=F_train_balanced)


/usr/local/lib/python3.7/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:275: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  FutureWarning,


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(learning_rate=1.0, max_depth=1),
                   prefit=True)

In [ ]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=F_test)

In [ ]:
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, F_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.574266,0.656472
Demographic parity difference,0.135897,0.047051
Demographic parity ratio,0.796154,0.929423
------,,
Overall balanced error rate,0.226029,0.245496
Balanced error rate difference,0.232173,0.247236
------,,
False positive rate difference,0.246095,0.343452
False negative rate difference,0.239216,0.166013
Equalized odds difference,0.246095,0.343452


# Mitigating Equalized Odds Difference in PAY1 feature  with Postprocessing algorithm ThresholdOptimizer

In [ ]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)

balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
F2_train_balanced = F2_train.loc[pp_train_idx]

In [ ]:
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=F2_train_balanced)


/usr/local/lib/python3.7/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:275: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  FutureWarning,


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(learning_rate=1.0, max_depth=1),
                   prefit=True)

In [ ]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=F2_test)
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, F2_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.574266,0.605639
Demographic parity difference,0.310653,0.208781
Demographic parity ratio,0.548603,0.667094
------,,
Overall balanced error rate,0.226029,0.279124
Balanced error rate difference,0.034601,0.024811
------,,
False positive rate difference,0.232776,0.205022
False negative rate difference,0.2586,0.176261
Equalized odds difference,0.2586,0.205022


# Mitigating Equalized Odds Difference in Female and Race feature  with Postprocessing algorithm ThresholdOptimizer

In [ ]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)

balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
A1_train_balanced = A1_train.loc[pp_train_idx]

In [ ]:
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=A1_train_balanced)


/usr/local/lib/python3.7/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:275: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  FutureWarning,


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(learning_rate=1.0, max_depth=1),
                   prefit=True)

In [ ]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=A1_test)
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, A1_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.574266,0.856145
Demographic parity difference,0.276653,0.101639
Demographic parity ratio,0.649573,0.887273
------,,
Overall balanced error rate,0.226029,0.421921
Balanced error rate difference,0.341945,0.241009
------,,
False positive rate difference,0.655263,0.384615
False negative rate difference,0.161265,0.097403
Equalized odds difference,0.655263,0.384615


# Mitigating Equalized Odds Difference in I10_NDX feature  with Postprocessing algorithm ThresholdOptimizer

In [ ]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)

balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
H_train_balanced = H_train.loc[pp_train_idx]

In [ ]:
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=H_train_balanced)

/usr/local/lib/python3.7/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:275: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  FutureWarning,


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(learning_rate=1.0, max_depth=1),
                   prefit=True)

In [ ]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=H_test)
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, H_test)

/usr/local/lib/python3.7/dist-packages/fairlearn/metrics/_metric_frame.py:515: RuntimeWarning: invalid value encountered in double_scalars
  result = self.group_min() / self.group_max()


,Unmitigated,ThresholdOptimizer
Overall selection rate,0.568236,0.0
Demographic parity difference,0.713534,0.0
Demographic parity ratio,0.152679,NaN
------,,
Overall balanced error rate,0.229062,0.5
Balanced error rate difference,0.226786,0.0
------,,
False positive rate difference,0.622711,0.0
False negative rate difference,0.714286,0.0
Equalized odds difference,0.714286,0.0


# After Using Threshhold Optimizer, we will implement Grid Search to mitigate Bias

In [ ]:
X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
A_train_balanced = A_train.loc[pp_train_idx]
B_train_balanced = B_train.loc[pp_train_idx]
C_train_balanced = C_train.loc[pp_train_idx]
D_train_balanced = D_train.loc[pp_train_idx]
E_train_balanced = E_train.loc[pp_train_idx]
F_train_balanced = F_train.loc[pp_train_idx]
F2_train_balanced = F2_train.loc[pp_train_idx]
A1_train_balanced = A1_train.loc[pp_train_idx]

In [ ]:
# General imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Data processing
from sklearn.model_selection import train_test_split

# # Models
# import lightgbm as lgb
# from sklearn.calibration import CalibratedClassifierCV

# Fairlearn algorithms and utils
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.reductions import GridSearch, EqualizedOdds

# Metrics
from fairlearn.metrics import (
    MetricFrame,
    selection_rate, demographic_parity_difference, demographic_parity_ratio,
    false_positive_rate, false_negative_rate,
    false_positive_rate_difference, false_negative_rate_difference,
    equalized_odds_difference)
from sklearn.metrics import balanced_accuracy_score, roc_auc_score

In [ ]:
# Train GridSearch
sweep = GridSearch(clf,
                   constraints=EqualizedOdds(),
                   grid_size=50,
                   grid_limit=3)

sweep.fit(X_train_balanced, y_train_balanced, sensitive_features=D_train_balanced)

In [ ]:
sweep_preds = [predictor.predict(X_test) for predictor in sweep.predictors_] 
sweep_scores = [predictor.predict_proba(X_test)[:, 1] for predictor in sweep.predictors_] 

In [ ]:
equalized_odds_sweep = [
    equalized_odds_difference(y_test, preds, sensitive_features=D_test)
    for preds in sweep_preds
]
balanced_accuracy_sweep = [balanced_accuracy_score(y_test, preds) for preds in sweep_preds]
auc_sweep = [roc_auc_score(y_test, scores) for scores in sweep_scores]

In [ ]:
# Select only non-dominated models (with respect to balanced accuracy and equalized odds difference)
all_results = pd.DataFrame(
    {"predictor": sweep.predictors_, "accuracy": balanced_accuracy_sweep, "disparity": equalized_odds_sweep}
) 
non_dominated = [] 
for row in all_results.itertuples(): 
    accuracy_for_lower_or_eq_disparity = all_results["accuracy"][all_results["disparity"] <= row.disparity] 
    if row.accuracy >= accuracy_for_lower_or_eq_disparity.max(): 
        non_dominated.append(True)
    else:
        non_dominated.append(False)

equalized_odds_sweep_non_dominated = np.asarray(equalized_odds_sweep)[non_dominated]
balanced_accuracy_non_dominated = np.asarray(balanced_accuracy_sweep)[non_dominated]
auc_non_dominated = np.asarray(auc_sweep)[non_dominated]

In [ ]:
# Compare GridSearch models with low values of equalized odds difference with the previously constructed models
grid_search_dict = {"GridSearch_{}".format(i): (sweep_preds[i], sweep_scores[i])
                    for i in range(len(sweep_preds))
                    if non_dominated[i] and equalized_odds_sweep[i]<0.1}
models_dict.update(grid_search_dict)
get_metrics_df(models_dict, y_test, D_test)

/usr/local/lib/python3.7/dist-packages/fairlearn/metrics/_metric_frame.py:515: RuntimeWarning: invalid value encountered in double_scalars
  result = self.group_min() / self.group_max()


,Unmitigated,ThresholdOptimizer,GridSearch_22,GridSearch_38
Overall selection rate,0.563994,0.617326,1.0,0.0
Demographic parity difference,0.308972,0.28005,0.0,0.0
Demographic parity ratio,0.620222,0.662797,1.0,NaN
------,,,,
Overall balanced error rate,0.228947,0.282129,0.5,0.5
Balanced error rate difference,0.136189,0.131317,0.0,0.0
------,,,,
False positive rate difference,0.4,0.39697,0.0,0.0
False negative rate difference,0.245843,0.205793,0.0,0.0
Equalized odds difference,0.4,0.39697,0.0,0.0


# EVALUATING UNFAIRNESS IN RANDOM FOREST ALGORITHM 


# RANDOM FOREST


In [ ]:
X=df[selected_features_arizona]
y = df['LOS_category']
A = df['FEMALE']
B = df['MARITALSTATUSUB04']
C = df['AGE']
D = df['RACE']
E = df['ZIPINC_QRTL']
F = df['MEDINCSTQ']
G = df['Homeless']
F2 = df['PAY1']
H = df['I10_NDX']
A1 = df[['FEMALE', 'RACE']]
# 'MARITALSTATUSUB04','PAY1','RACE','ZIPINC_QRTL'
X_train, X_test, y_train, y_test, A_train, A_test, B_train, B_test,C_train, C_test, D_train, D_test, E_train, E_test, F_train, F_test, G_train, G_test, H_train,H_test, F2_train,F2_test, A1_train,A1_test, = train_test_split(df[selected_features_arizona], 
                                                                     df['LOS_category'],
                                                                     A,B,C,D,E,F,G,H,F2,A1,
                                                                     stratify=df['LOS_category'],
                                                                     test_size=0.3)

In [ ]:

forest = RandomForestClassifier(criterion='gini',
                                 n_estimators=5,
                                 random_state=1,
                                 n_jobs=2)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

Accuracy: 0.754


In [ ]:
from sklearn.metrics import classification_report
target_names=['0','1']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           0       0.68      0.63      0.65     19202
           1       0.79      0.82      0.81     32750

    accuracy                           0.75     51952
   macro avg       0.74      0.73      0.73     51952
weighted avg       0.75      0.75      0.75     51952



In [ ]:
test_scores = forest.predict_proba(X_test)[:, 1]
test_preds = (test_scores >= np.mean(y_train)) * 1
# Train AUC
roc_auc_score(y_train, forest.predict_proba(X_train)[:, 1])

0.9940232695849662

# EVALUATING UNFAIRNESS OF RANDOM FOREST MODEL IN DEMOGRAPHIC FEATURES

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=A_test)

mf.by_group

,FPR,FNR
FEMALE,,
0,0.211207,0.321325
1,0.188947,0.349772


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, A_test)

,Unmitigated
Overall selection rate,0.49053
Demographic parity difference,0.040767
Demographic parity ratio,0.920772
------,
Overall balanced error rate,0.267688
Balanced error rate difference,0.003093
------,
False positive rate difference,0.022261
False negative rate difference,0.028447
Equalized odds difference,0.028447


# MARITAL STATUS


In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=B_test)

mf.by_group

,FPR,FNR
MARITALSTATUSUB04,,
0,0.21791,0.371304
1,0.255465,0.293471
2,0.138528,0.325203
3,0.186047,0.345815
4,0.172073,0.349453
5,0.240154,0.362609


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, B_test)

,Unmitigated
Overall selection rate,0.49053
Demographic parity difference,0.108597
Demographic parity ratio,0.809076
------,
Overall balanced error rate,0.267688
Balanced error rate difference,0.069516
------,
False positive rate difference,0.116937
False negative rate difference,0.077833
Equalized odds difference,0.116937


# AGE 


In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=C_test)

mf.by_group

,FPR,FNR
AGE,,
0,0.11928,0.391002
1,0.206467,0.333443


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, C_test)

,Unmitigated
Overall selection rate,0.49053
Demographic parity difference,0.107594
Demographic parity ratio,0.784649
------,
Overall balanced error rate,0.267688
Balanced error rate difference,0.014814
------,
False positive rate difference,0.087187
False negative rate difference,0.057559
Equalized odds difference,0.087187


#RACE

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=D_test)

mf.by_group

,FPR,FNR
RACE,,
0,0.152778,0.372832
1,0.182745,0.350128
2,0.186671,0.3382
3,0.157534,0.32
4,0.207305,0.323928
5,0.307692,0.137931
6,0.202086,0.337405


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, D_test)

,Unmitigated
Overall selection rate,0.49053
Demographic parity difference,0.245636
Demographic parity ratio,0.644251
------,
Overall balanced error rate,0.267688
Balanced error rate difference,0.046934
------,
False positive rate difference,0.154915
False negative rate difference,0.234901
Equalized odds difference,0.234901


#ZIPINC_QTRL

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=E_test)

mf.by_group

,FPR,FNR
ZIPINC_QRTL,,
0,0.193743,0.337708
1,0.210301,0.330626
2,0.193463,0.327901
3,0.202254,0.350302
4,0.16568,0.393191


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, E_test)

,Unmitigated
Overall selection rate,0.49053
Demographic parity difference,0.07309
Demographic parity ratio,0.854353
------,
Overall balanced error rate,0.267688
Balanced error rate difference,0.018754
------,
False positive rate difference,0.044621
False negative rate difference,0.06529
Equalized odds difference,0.06529


#MEDINCSTQ

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=F_test)

mf.by_group

,FPR,FNR
MEDINCSTQ,,
0,0.192308,0.339197
1,0.203313,0.331649
2,0.209429,0.326017
3,0.194539,0.344917
4,0.25,1.0
5,0.165347,0.391973


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, F_test)

,Unmitigated
Overall selection rate,0.49053
Demographic parity difference,0.361246
Demographic parity ratio,0.283389
------,
Overall balanced error rate,0.267688
Balanced error rate difference,0.359248
------,
False positive rate difference,0.084653
False negative rate difference,0.673983
Equalized odds difference,0.673983


#Homeless

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=G_test)

mf.by_group

,FPR,FNR
Homeless,,
0,0.147388,0.44186
1,0.199079,0.335791


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, G_test)

,Unmitigated
Overall selection rate,0.49053
Demographic parity difference,0.128673
Demographic parity ratio,0.739183
------,
Overall balanced error rate,0.267688
Balanced error rate difference,0.02719
------,
False positive rate difference,0.05169
False negative rate difference,0.10607
Equalized odds difference,0.10607


#PAY1

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=F2_test)

mf.by_group

,FPR,FNR
PAY1,,
0,0.211833,0.36002
1,0.192024,0.328677
2,0.183277,0.330194
3,0.218,0.272512
4,0.179487,0.416667
5,0.2,0.358508


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, F2_test)

,Unmitigated
Overall selection rate,0.49053
Demographic parity difference,0.127954
Demographic parity ratio,0.780614
------,
Overall balanced error rate,0.267688
Balanced error rate difference,0.052821
------,
False positive rate difference,0.038513
False negative rate difference,0.144155
Equalized odds difference,0.144155


#I10_NDX

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=H_test)

mf.by_group

,FPR,FNR
I10_NDX,,
0,0.333333,0.517241
1,0.1742,0.29364
2,0.207216,0.28125
3,0.193021,0.293609
4,0.238624,0.280299
5,0.249683,0.29169
6,0.26259,0.307147
7,0.263158,0.328158
8,0.279601,0.334601


In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, H_test)

,Unmitigated
Overall selection rate,0.49053
Demographic parity difference,0.471611
Demographic parity ratio,0.168013
------,
Overall balanced error rate,0.267688
Balanced error rate difference,0.332432
------,
False positive rate difference,0.244048
False negative rate difference,0.729188
Equalized odds difference,0.729188


#FEMALE AND RACE

In [ ]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=A1_test)

mf.by_group

FPR       FNR
FEMALE RACE                    
0      0     0.155844  0.318182
       1     0.210832  0.330199
       2     0.192623  0.310783
       3     0.142857  0.256637
       4     0.215584  0.322302
       5          0.4  0.166667
       6     0.215458   0.32335
1      0     0.151079  0.406542
       1     0.167539  0.363899
       2     0.183824   0.35416
       3     0.164948  0.353774
       4     0.202229  0.324977
       5         0.25  0.130435
       6      0.19273  0.348533

In [ ]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, A1_test)

,Unmitigated
Overall selection rate,0.49053
Demographic parity difference,0.290414
Demographic parity ratio,0.59078
------,
Overall balanced error rate,0.267688
Balanced error rate difference,0.093116
------,
False positive rate difference,0.257143
False negative rate difference,0.276107
Equalized odds difference,0.276107


#MITIGATING BIAS IN FEATURES FOR RANDOM FOREST 

#RACE

In [ ]:
postprocess_est = ThresholdOptimizer(
    estimator=forest,
    constraints="equalized_odds",
    prefit=True)

balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
D_train_balanced = D_train.loc[pp_train_idx]

In [ ]:
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=D_train_balanced)
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=D_test)
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, D_test)

/usr/local/lib/python3.7/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:275: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  FutureWarning,


,Unmitigated,ThresholdOptimizer
Overall selection rate,0.49053,0.668983
Demographic parity difference,0.245636,0.146924
Demographic parity ratio,0.644251,0.813005
------,,
Overall balanced error rate,0.267688,0.283316
Balanced error rate difference,0.046934,0.087341
------,,
False positive rate difference,0.154915,0.209694
False negative rate difference,0.234901,0.095974
Equalized odds difference,0.234901,0.209694


In [ ]:
postprocess_est = ThresholdOptimizer(
    estimator=forest,
    constraints="equalized_odds",
    prefit=True)

balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
F_train_balanced = F_train.loc[pp_train_idx]

In [ ]:
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=F_train_balanced)
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=F_test)
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, F_test)

/usr/local/lib/python3.7/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:275: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  FutureWarning,


,Unmitigated,ThresholdOptimizer
Overall selection rate,0.49053,0.6578
Demographic parity difference,0.361246,0.241337
Demographic parity ratio,0.283389,0.639746
------,,
Overall balanced error rate,0.267688,0.274011
Balanced error rate difference,0.359248,0.313516
------,,
False positive rate difference,0.084653,0.145545
False negative rate difference,0.673983,0.497494
Equalized odds difference,0.673983,0.497494


In [ ]:
postprocess_est = ThresholdOptimizer(
    estimator=forest,
    constraints="equalized_odds",
    prefit=True)

balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
F2_train_balanced = F2_train.loc[pp_train_idx]

In [ ]:
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=F2_train_balanced)
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=F2_test)
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, F2_test)

/usr/local/lib/python3.7/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:275: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  FutureWarning,


,Unmitigated,ThresholdOptimizer
Overall selection rate,0.49053,0.658088
Demographic parity difference,0.127954,0.142065
Demographic parity ratio,0.780614,0.806862
------,,
Overall balanced error rate,0.267688,0.2737
Balanced error rate difference,0.052821,0.064305
------,,
False positive rate difference,0.038513,0.07803
False negative rate difference,0.144155,0.161758
Equalized odds difference,0.144155,0.161758


In [ ]:
postprocess_est = ThresholdOptimizer(
    estimator=forest,
    constraints="equalized_odds",
    prefit=True)

balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
A1_train_balanced = A1_train.loc[pp_train_idx]

In [ ]:
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=A1_train_balanced)
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=A1_test)
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, A1_test)

/usr/local/lib/python3.7/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:275: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  FutureWarning,


,Unmitigated,ThresholdOptimizer
Overall selection rate,0.49053,0.74232
Demographic parity difference,0.290414,0.192377
Demographic parity ratio,0.59078,0.788385
------,,
Overall balanced error rate,0.267688,0.335848
Balanced error rate difference,0.093116,0.130687
------,,
False positive rate difference,0.257143,0.330612
False negative rate difference,0.276107,0.17757
Equalized odds difference,0.276107,0.330612


In [ ]:
postprocess_est = ThresholdOptimizer(
    estimator=forest,
    constraints="equalized_odds",
    prefit=True)

balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
H_train_balanced = H_train.loc[pp_train_idx]

In [ ]:
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=H_train_balanced)
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=H_test)
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, H_test)

/usr/local/lib/python3.7/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:275: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  FutureWarning,


,Unmitigated,ThresholdOptimizer
Overall selection rate,0.49053,0.629061
Demographic parity difference,0.471611,0.458934
Demographic parity ratio,0.168013,0.341586
------,,
Overall balanced error rate,0.267688,0.28809
Balanced error rate difference,0.332432,0.375958
------,,
False positive rate difference,0.244048,0.241236
False negative rate difference,0.729188,0.828151
Equalized odds difference,0.729188,0.828151


In [ ]:
# # Unfairness mitigation
# # To do: 
# # 1. Add metric computation after each model training. 
# # 1. Add save to file after each metric computation. 
# # 2. Run and check the results. 

# # Set the model to which the model mitigation technique needs to be applied. 
# estimator = RandomForestClassifier
# mitigated_result = pd.DataFrame
# def unfairness_mitigation(model_to_train, X_train, y_train, X_test, y_test):
#     """
#     Returns a list of models which are mitigated.  
#     """
#     # To do: 
#     # 1. Add save results to csv. 
#     mitigated_models = []
#     mitigated = ThresholdOptimizer(estimator=RandomForestClassifier(), constraints='equalized_odds',
#                                    objective='accuracy_score', grid_size=1000,
#                                    flip=False, prefit=False,
#                                    predict_method='deprecated')
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['ZIPINC_QRTL']])
#     y_pred = mitigated.predict(X_test, sensitive_features=X_train[['ZIPINC_QRTL']])
#     mitigated_models.append(mitigated)

#     mitigated_models.append(mitigated)
#     unfairness_metrics_evaluation([mitigated], X_test, y_test)
    
#     mitigated = ThresholdOptimizer(estimator=RandomForestClassifier(), constraints='demographic_parity',
#                                    objective='accuracy_score', grid_size=1000,
#                                    flip=False, prefit=False,
#                                    predict_method='deprecated')
#     mitigated.fit(X_train, y_train)
#     y_pred = mitigated.predict(X_test)

#     mitigated_models.append(mitigated)
#     unfairness_metrics_evaluation([mitigated], X_test, y_test)

#     moment = DemographicParity()
#     mitigated = ExponentiatedGradient(estimator=RandomForestClassifier(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['ZIPINC_QRTL']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     moment = DemographicParity()
#     mitigated = ExponentiatedGradient(estimator=RandomForestClassifier(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['RACE']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)


#     moment = DemographicParity()
#     mitigated = ExponentiatedGradient(estimator=RandomForestClassifier(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['AGE']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)


#     moment = DemographicParity()
#     mitigated = ExponentiatedGradient(estimator=RandomForestClassifier(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['FEMALE']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     moment = DemographicParity()
#     mitigated = ExponentiatedGradient(estimator=RandomForestClassifier(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['PAY1_X']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     moment = DemographicParity()
#     mitigated = GridSearch(estimator=RandomForestClassifier(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['ZIPINC_QRTL']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     moment = DemographicParity()
#     mitigated = GridSearch(estimator=RandomForestClassifier(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['RACE']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)


#     moment = DemographicParity()
#     mitigated = GridSearch(estimator=RandomForestClassifier(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['AGE']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     moment = DemographicParity()
#     mitigated = GridSearch(estimator=RandomForestClassifier(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['FEMALE']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     moment = DemographicParity()
#     mitigated = GridSearch(estimator=RandomForestClassifier(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['PAY1_X']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     unfairness_metrics_evaluation(mitigated_models, X_test, y_test, prefix="mitigated")
    
#     return mitigated_models


In [ ]:
# # Unfairness mitigation
# # To do: 
# # 1. Add metric computation after each model training. 
# # 1. Add save to file after each metric computation. 
# # 2. Run and check the results. 

# # Set the model to which the model mitigation technique needs to be applied. 
# m = RandomForestClassifier
# mitigated_result = pd.DataFrame
# mitigated_models = []
# def unfairness_mitigation(model_to_train, X_train, y_train, X_test, y_test):
#     """
#     Returns a list of models which are mitigated.  
#     """
#     # To do: 
#     # 1. Add save results to csv. 
    
#     mitigated = ThresholdOptimizer(estimator=m(), constraints='equalized_odds',
#                                    objective='accuracy_score', grid_size=1000,
#                                    flip=False, prefit=False,
#                                    predict_method='predict') #deprecated
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['ZIPINC_QRTL']])
#     y_pred = mitigated.predict(X_test, sensitive_features=X_test[['ZIPINC_QRTL']] )
#     # mitigated_models.append(mitigated)
#     # return mitigated_models

#     mitigated_models.append(mitigated)
#     # unfairness_metrics_evaluation([mitigated], X_test, y_test)
    
#     mitigated = ThresholdOptimizer(estimator=m(), constraints='demographic_parity',
#                                    objective='accuracy_score', grid_size=1000,
#                                    flip=False, prefit=False,
#                                    predict_method='predict')
#     x=mitigated.fit(X_train, y_train, sensitive_features=X_train[['ZIPINC_QRTL']])
#     y_pred = mitigated.predict(X_test, sensitive_features=X_test[['ZIPINC_QRTL']])
#     mitigated_models.append(x)
#     unfairness_metrics_evaluation([mitigated], X_test, y_test)

#     moment = DemographicParity()
#     mitigated = ExponentiatedGradient(estimator=m(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['ZIPINC_QRTL']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     moment = DemographicParity()
#     mitigated = ExponentiatedGradient(estimator=m(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['RACE']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)


#     moment = DemographicParity()
#     mitigated = ExponentiatedGradient(estimator=m(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['AGE']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)


#     moment = DemographicParity()
#     mitigated = ExponentiatedGradient(estimator=m(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['FEMALE']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     moment = DemographicParity()
#     mitigated = ExponentiatedGradient(estimator=m(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['PAY1_X']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     moment = DemographicParity()
#     mitigated = GridSearch(estimator=m(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['ZIPINC_QRTL']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     moment = DemographicParity()
#     mitigated = GridSearch(estimator=m(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['RACE']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)


#     moment = DemographicParity()
#     mitigated = GridSearch(estimator=m(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['AGE']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     moment = DemographicParity()
#     mitigated = GridSearch(estimator=m(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['FEMALE']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     moment = DemographicParity()
#     mitigated = GridSearch(estimator=m(), constraints=moment)
#     mitigated.fit(X_train, y_train, sensitive_features=X_train[['PAY1_X']])
#     y_pred = mitigated.predict(X_test)
#     mitigated_models.append(mitigated)

#     unfairness_metrics_evaluation(mitigated_models, X_test, y_test, prefix="mitigated")
    
#     return mitigated_models


In [ ]:
[print(mitigated_models)]

[]


[None]

In [ ]:
mitigated_models = unfairness_mitigation(models_to_train, X_train[selected_features_arizona], y_train, X_test[selected_features_arizona], y_test)

ValueError: ignored

In [ ]:
# mitigated_models = unfairness_mitigation(models_to_train, X_train[selected_features_arizona], y_train, X_test[selected_features_arizona], y_test)

TypeError: ignored

In [ ]:
len(X_test[selected_features_arizona])

51958

In [ ]:
len(y_test)

51958

In [ ]:
import pytorch_tabnet
from pytorch_tabnet.tab_model importTabNetClassifier
import torch
 
from sklearn.preprocessing importLabelEncoder
from sklearn.metrics importroc_auc_score, accuracy_score

In [ ]:
# TABNET MODEL
tabnet_model = TabNetClassifier(optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params={"step_size":10, # how to use learning rate scheduler
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax' # "sparsemax"
                      )
 
# fit the model 
tabnet_model.fit(
    X_train.values,y_train.values,
    eval_set=[(X_train.values,y_train.values), (X_test.values,y_test.values)],
    eval_name=['train', 'valid'],
    eval_metric=['auc','accuracy'],
    max_epochs=50 , patience=50,
    batch_size=256,virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
)

unfairness_metrics_evaluation([tabnet_model], X_test, y_test)